In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split 
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, Concatenate, TimeDistributed
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.keras import mixed_precision

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-05-22 12:43:54.079406: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 12:43:54.730880: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-22 12:43:55.887490: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-05-22 12:43:55.887525: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: Nitro
2024-05-22 12:43:5

In [2]:
data = pd.read_csv('scaled_training.csv')
data = data.iloc[:200000,:]
display(data)

,date-time,Microwave,General,Refrigerator,Washing Machine,Shower,Air Fryer
0,2022-10-03 18:43:45,0.0,0.007063,0.000000,0.0,0.0,0.0
1,2022-10-03 18:43:50,0.0,0.007219,0.000000,0.0,0.0,0.0
2,2022-10-03 18:43:55,0.0,0.007212,0.000000,0.0,0.0,0.0
3,2022-10-03 18:44:00,0.0,0.007201,0.000000,0.0,0.0,0.0
4,2022-10-03 18:44:05,0.0,0.007160,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...
199995,2022-10-15 10:33:00,0.0,0.028901,0.170134,0.0,0.0,0.0
199996,2022-10-15 10:33:05,0.0,0.028857,0.169902,0.0,0.0,0.0
199997,2022-10-15 10:33:10,0.0,0.028873,0.169791,0.0,0.0,0.0
199998,2022-10-15 10:33:15,0.0,0.028820,0.169679,0.0,0.0,0.0


In [3]:
for col in data.columns :
    print("Unique Values in " + col + " :" , data[col].nunique())

Unique Values in date-time : 200000
Unique Values in Microwave : 1
Unique Values in General : 39382
Unique Values in Refrigerator : 5008
Unique Values in Washing Machine : 542
Unique Values in Shower : 609
Unique Values in Air Fryer : 1


In [4]:
#General Power Consumption Signal as Input
x = data['General'].to_numpy()
  
# y are the Individual Appliance Signal which are to be predicted
y = data.loc[:, ['Refrigerator','Washing Machine','Shower']].to_numpy()
  
# Splitting dataset in 70-15-15 fashion for Training,Testing and Validation DataSets
x_train, x_Combine, y_train, y_Combine = train_test_split(x,y,test_size=0.3,random_state=25)
x_val, x_test, y_val, y_test = train_test_split(x_Combine,y_Combine,test_size=0.5,random_state=42) 

print(x_train , y_train)
print(x_val , y_val)
print(x_test , y_test)

[0.01008266 0.01287256 0.03321268 ... 0.03641411 0.03180699 0.02645533] [[0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.17319069 0.         0.        ]
 ...
 [0.17197146 0.         0.        ]
 [0.17149064 0.         0.        ]
 [0.1688547  0.         0.        ]]
[0.04206628 0.00768956 0.01043592 ... 0.02872209 0.04307455 0.01004096] [[0.17137902 0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 ...
 [0.16929259 0.         0.        ]
 [0.17204015 0.         0.        ]
 [0.         0.         0.        ]]
[0.01841987 0.03070918 0.0304332  ... 0.0115325  0.03217497 0.04624041] [[0.         0.         0.        ]
 [0.16879459 0.         0.        ]
 [0.17298462 0.         0.        ]
 ...
 [0.         0.         0.        ]
 [0.17511398 0.         0.        ]
 [0.16985069 0.         0.        ]]


In [5]:
print(x_train.shape , y_train.shape)

(140000,) (140000, 3)


In [6]:
# Parameters
window_size = 100  # Number of time steps in each window

# Create sliding windows
def create_sliding_windows(data, window_size):
    windows = []
    for i in range(len(data) - window_size + 1):
        windows.append(data[i:i+window_size])
    return np.array(windows)

x_train = create_sliding_windows(x_train, window_size)
x_train = np.expand_dims(x_train, axis=2)  # Add feature dimension

# Example shapes
print("Input shape:", x_train.shape)  # (number_of_windows, window_size, 1)

Input shape: (139901, 100, 1)


In [7]:
x_val = create_sliding_windows(x_val, window_size)
x_val = np.expand_dims(x_val, axis=2)
x_test = create_sliding_windows(x_test, window_size)
x_test = np.expand_dims(x_test, axis=2) 

In [8]:
print(x_val.shape)  # (number_of_windows, window_size, 1)

(29901, 100, 1)


In [9]:
print(y_train.shape)

(140000, 3)


In [10]:
def create_sliding_windows_y(data, window_size):
    num_windows = len(data) - window_size + 1
    shape = (num_windows, window_size, data.shape[1])
    strides = (data.strides[0], data.strides[0], data.strides[1])
    windows = np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)
    return windows

In [11]:
y_train = create_sliding_windows_y(y_train, window_size)
print("Target shape:", y_train.shape)  # (number_of_windows, window_size, 4)

Target shape: (139901, 100, 3)


In [12]:
y_val = create_sliding_windows_y(y_val, window_size)
print("Target shape:", y_val.shape)  # (number_of_windows, window_size, 4)

Target shape: (29901, 100, 3)


In [13]:
y_test = create_sliding_windows_y(y_test, window_size)
print("Target shape:", y_test.shape)  # (number_of_windows, window_size, 4)

Target shape: (29901, 100, 3)


In [14]:
# Define the Attention layer
# class AttentionLayer(tf.keras.layers.Layer):
#     def __init__(self, **kwargs):
#         super(AttentionLayer, self).__init__(**kwargs)

#     def build(self, input_shape):
#         self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
#                                  initializer='random_normal', trainable=True)
#         self.b = self.add_weight(shape=(input_shape[-1],),
#                                  initializer='random_normal', trainable=True)
#         self.V = self.add_weight(shape=(input_shape[-1], 1),
#                                  initializer='random_normal', trainable=True)
#         super(AttentionLayer, self).build(input_shape)

#     def call(self, inputs):
#         score = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
#         attention_weights = tf.nn.softmax(tf.tensordot(score, self.V, axes=1), axis=1)
#         context_vector = attention_weights * inputs
#         context_vector = tf.reduce_sum(context_vector, axis=1)
#         return context_vector, attention_weights

class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], input_shape[-1]), initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[-1],), initializer='zeros', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        score = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

class ExpandDimsLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.expand_dims(inputs, axis=1)

    
def create_model(input_shape, num_outputs):
    inputs = Input(shape=input_shape)

    # Encoder
    encoder_lstm = LSTM(128, return_sequences=True, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(inputs)

    # Attention
    context_vector, attention_weights = AttentionLayer()(encoder_outputs)
    
    # ExpandDims Layer
    context_vector = ExpandDimsLayer()(context_vector)
    
    # Decoder
    decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(context_vector, initial_state=[state_h, state_c])

    outputs = TimeDistributed(Dense(num_outputs ,activation='relu'))(decoder_outputs)  # Ensure correct output shape

    model = Model(inputs, outputs)
    return model

# Define window size, number of features and number of outputs
window_size = 100
num_features = 1  # Assuming 1 feature for total power
num_outputs = 3  # Assuming 3 appliances


policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Create the model
input_shape = (window_size, num_features)
model = create_model(input_shape, num_outputs)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, x, y, batch_size):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.x))

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x = self.x[batch_indexes]
        batch_y = self.y[batch_indexes]
        return batch_x, batch_y

# Ensure data shapes are correct
x_train = x_train.reshape(-1, window_size, num_features)
y_train = y_train.reshape(-1, window_size, num_outputs)
x_val = x_val.reshape(-1, window_size, num_features)
y_val = y_val.reshape(-1, window_size, num_outputs)
x_test = x_test.reshape(-1, window_size, num_features)
y_test = y_test.reshape(-1, window_size, num_outputs)

# Create data generators
train_generator = DataGenerator(x_train, y_train, batch_size=32)
val_generator = DataGenerator(x_val, y_val, batch_size=32)
test_generator = DataGenerator(x_test, y_test, batch_size=32)


checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_checkpoint.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min'
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs')

#Train the model with callbacks
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=val_generator,
                    callbacks=[checkpoint_callback, tensorboard_callback])

# Train the model
#history = model.fit(train_generator, epochs=50, validation_data=val_generator)

# Evaluate the model
test_loss, test_mae = model.evaluate(test_dataset)
print(f'Test MAE: {test_mae}')

# Predict individual appliance consumption
y_pred = model.predict(test_dataset)
print(y_pred)


Epoch 1/50
   1/4372 ━━━━━━━━━━━━━━━━━━━━ 2:25:42 2s/step - loss: 0.0045 - mae: 0.0263

/home/adeem/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4372/4372 ━━━━━━━━━━━━━━━━━━━━ 264s 60ms/step - loss: 0.0033 - mae: 0.0293 - val_loss: 0.0034 - val_mae: 0.0295
Epoch 2/50
4372/4372 ━━━━━━━━━━━━━━━━━━━━ 263s 60ms/step - loss: 0.0033 - mae: 0.0293 - val_loss: 0.0034 - val_mae: 0.0294
Epoch 3/50
4372/4372 ━━━━━━━━━━━━━━━━━━━━ 261s 60ms/step - loss: 0.0033 - mae: 0.0293 - val_loss: 0.0034 - val_mae: 0.0295
Epoch 4/50
4372/4372 ━━━━━━━━━━━━━━━━━━━━ 268s 61ms/step - loss: 0.0033 - mae: 0.0293 - val_loss: 0.0034 - val_mae: 0.0295
Epoch 5/50
4372/4372 ━━━━━━━━━━━━━━━━━━━━ 269s 62ms/step - loss: 0.0033 - mae: 0.0293 - val_loss: 0.0034 - val_mae: 0.0294
Epoch 6/50
4372/4372 ━━━━━━━━━━━━━━━━━━━━ 266s 61ms/step - loss: 0.0033 - mae: 0.0293 - val_loss: 0.0034 - val_mae: 0.0295
Epoch 7/50
4372/4372 ━━━━━━━━━━━━━━━━━━━━ 265s 61ms/step - loss: 0.0034 - mae: 0.0293 - val_loss: 0.0034 - val_mae: 0.0293
Epoch 8/50
4372/4372 ━━━━━━━━━━━━━━━━━━━━ 263s 60ms/step - loss: 0.0033 - mae: 0.0292 - val_loss: 0.0034 - val_mae: 0.0293
Epoch 9/50
4372/4372 ━━━━━━

In [ ]:
model.save('updated_model.keras')

In [ ]:
print(y_pred.shape)

In [ ]:
pred_df = pd.DataFrame(new_y_pred)

In [ ]:
display(pred_df)

In [ ]:
y_test_df = pd.DataFrame(y_test)
display(y_test_df)

In [ ]:
for col in pred_df.columns:
    plt.figure(figsize=(16, 8), dpi=150)
    pred_df[col].head(50000).plot(label=col, color='red')
    plt.title('Power Consumption Plot')
    plt.xlabel('TimeStamp')
    plt.ylabel(col)
    plt.legend()
    plt.show()

In [ ]:
for col in y_test_df.columns:
    plt.figure(figsize=(16, 8), dpi=150)
    y_test_df[col].head(50000).plot(label=col, color='green')
    plt.title('Power Consumption Plot')
    plt.xlabel('TimeStamp')
    plt.ylabel(col)
    plt.legend()
    plt.show()